In [ ]:
%autosave 5
import yaml
import pandas as pd

# Directory structure
    code / __init__.py
         / setup.py()
         / job      / __init__.py
                    / {\*yml, \*py}
         / library1 / __init__.py
                    / {\*py}
         :
         / libraryN

Directory structure
1. Root directory (currently named `code`) containing
  a) `__init__.py` file 
  b) `setup.py` file 
2. `job` directory with main script(s) and possibly yaml configuration files. Needs to have `__init__.py` file
3. libraries (may have arbitrary names) with python code only and containging the `__init__.py`

# SETUP.PY

Uses setuputils to prepare a packege by the `gcloud` command

AI Jobs instances provide a reange of python libraries including the google suite, but sometimes additional are needed. Those should be specified in _REQUIRED\_PACKAGES_. Do note that versions are important to satisfy all the dependencies with google suite modules, especially, not always the newest versions work. You may inspect the confilcts in the AI Jobs logs

_name_, _version_, and _description_ - may be changed as needed

In [220]:
%%writefile code/setup.py

# creating package for AI JOBS environment

from setuptools import find_packages
from setuptools import setup

# specify required packages and versions
# some recent version of python packages doesn't work with google packages
REQUIRED_PACKAGES = ['google-cloud-bigquery', 'google-cloud-bigquery-storage','scikit-learn>=0.24','numpy==1.19.5','pandas==1.3','statsmodels==0.12.2','pmdarima']
# scikit-learn is to old by default and we need 0.24 at least
# numpy 1.19.5 is necessary for pandas binding with pyarrow (to work with bigquery). The later version 1.20 doesn't work well with pyarrow
# pandas - not sure if necessary
# pmdarima - necessary for poc model

setup(
  name='chargeable-demand-train-package',
  version='0.1',
  install_requires=REQUIRED_PACKAGES,
  packages=find_packages(),  # automatically find suporting modules like library1
  description='A deployment package for training on Cloud ML Engine.',
#  scripts = ['job/main.py','job/config.yml','job/config.py',] , #'job/bq.py' # This is not necessary
  package_data = {'job' : ["*.yml", "job/*"] }  # required so the yaml fiels are included in the package and sent to the instance
)

Overwriting code/setup.py


# yaml config file
holds information used by the job
can be edited by hand or automatically by the batch submission script

In [4]:
config = {
    "PROJECT" : 'prd-65343-datalake-bd-88394358',
    "DATASET" : 'hrdemandforecast_9375_ds',
    "target" : "Chargeable_FTE",
}

with open('code/job/config.yml','w') as _f:
    yaml.dump(config, _f)

# creating main job file
The same code won't run in the notebook. For tests you can execute the script created by this code inside the job directory. FOr example `python example.py`

In [ ]:
%%writefile code/job/example.py
# the code must be located in code/job/ directory


#############################################
#### Identification of the working directory
#### Needed for accessing the other files through direct paths

import os
jobdir = os.path.dirname(os.path.realpath(__file__))
print('jobdir',jobdir)

#############################################

#### The rest of the code can be addapted for the needs
#  read libraries relative to the code directory not the script working directory
from library1.example_module import example_func



def main():

    # loading the config file (path needs to be absolute)
    with open(jobdir+'/config.yml','r') as _f:
        config = yaml.safe_load(_f)

    # You can use prints for you logging messages
    print("This message will appear as a INFO level log in AI Jobs logging information")
    
    # Write results to BQ or GCS. All files and temporary results will be lost at the end of the job!!!

    
if __name__ == '__main__':
    main()

# Simple submit job

In [379]:
# Running dataprep jobs
from datetime import datetime
from os import system


timestamp = datetime.utcnow().strftime("%y%m%d%H%M%S%f")  # just to make every job name unique
job_name=f'{intersection}_{timestamp}'
package_path="code/job"  # related to directory structure
module_name="job.example"  # TODO change 'example' to the name of your job script

job_submission_cmd = f"""
gcloud ai-platform jobs submit training "{job_name_prefix}{job_name}" \
    --staging-bucket gs://prd-9375-chargdemand-ds-staging \
    --job-dir gs://prd-9375-chargdemand-ds-pkgs/"charge_demand_job"_"{job_name}" \
    --python-version=3.7 \
    --runtime-version=2.2 \
    --module-name {module_name} \
    --package-path {package_path} \
    --region us-central1 \
    --scale-tier BASIC \
#    --packages='gs://prd-9375-chargdemand-ds-pkgs/pypackages/gstudio-0.0.20.tar.gz'
"""

ret = system(job_submission_cmd)
if ret!=0:
    print(f"Nonezero submission status ({ret})!!!")

CPU times: user 12 µs, sys: 5 µs, total: 17 µs
Wall time: 19.1 µs


# Batch submit job using aijobslib

In [ ]:
# submit multiple jobs for intersections
from os import system

def submit_jobs(intersections, module_name, job_name_prefix="charge_demand_aijob_", labels=None):
    
    job_names = []
    for intersection in intersections:

        print(intersection)

        # updating the configuration file to the new intersection
        with open('code/job/poc_model_config.yml','r') as _f:
            config = yaml.safe_load(_f)
        config['intersection'] = intersection    
        with open('code/job/poc_model_config.yml','w') as _f:
            yaml.dump(config, _f)



        timestamp = datetime.datetime.utcnow().strftime("%y%m%d%H%M%S%f")  # just to make every job name unique
        job_name=f'{intersection}_{timestamp}'
        package_path="code/job"  # related to directory structure

        # the **{} is necessary due to hythens in option names (cannot be simply used as function arguments)
        ret = aij.submit_training(f"{job_name_prefix}{job_name}", **{"module-name":module_name, "package-path":package_path})
        if ret!=0:
            print(f"Nonezero submission status ({ret})!!!")
        else:
            job_names.append(job_name_prefix+job_name)
    
    return job_names